<a href="https://colab.research.google.com/github/huysn910hn/Fine-tuning-PhoBERT-sentiment-classfication-with-tensorflow/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import TFAutoModel
from keras.saving import register_keras_serializable
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [18]:
df = pd.read_csv('data.csv')
df.head()

df.info()
print(df.columns)
df['label'].value_counts()
print(df['comment'][1])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16175 entries, 0 to 16174
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   comment       16175 non-null  object
 1   label         16175 non-null  int64 
 2   cleaned_text  16175 non-null  object
dtypes: int64(1), object(2)
memory usage: 379.2+ KB
Index(['comment', 'label', 'cleaned_text'], dtype='object')
nhiệt tình giảng dạy gần gũi với sinh viên


In [19]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [20]:
import pandas as pd
import re
letters = set('aáàảãạăắằẳẵặâấầẩẫậbcdđeéèẻẽẹêếềểễệfghiíìỉĩịjklmnoóòỏõọôốồổỗộơớờởỡợpqrstuúùủũụưứừửữựvwxyýỳỷỹỵz0123456789')

df = df.dropna(subset=['comment'])
df = df[df['comment'].str.strip() != '']
df = df.reset_index(drop = True)
df.to_csv('data.csv', index = False)

def clean_word(w):
    return ''.join(letter.lower() for letter in w if letter.lower() in letters)

def preprocess_text(text):
    if pd.isna(text):
        return ""

    text = re.sub(r"[^\w\s]", " ", text)
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    words = text.split()
    cleaned_words = [clean_word(word) for word in words]

    return " ".join(cleaned_words)

def preprocess_csv(file_path, text_column="comment", output_file="data.csv"):
    df = pd.read_csv(file_path)
    df["comment"] = df[text_column].astype(str).apply(preprocess_text)
    df.to_csv(output_file, index=False)

    return df


df = preprocess_csv("data.csv", text_column="comment")
print(df[["comment"]].head())


                                             comment
0                            slide giáo trình đầy đủ
1         nhiệt tình giảng dạy gần gũi với sinh viên
2                 đi học đầy đủ full điểm chuyên cần
3  chưa áp dụng công nghệ thông tin và các thiết ...
4  thầy giảng bài hay có nhiều bài tập ví dụ ngay...


In [21]:
print(df['comment'][2])

đi học đầy đủ full điểm chuyên cần


In [22]:
token = tokenizer.encode_plus(
    df['comment'][2],
    max_length=256,
    truncation=True,
    padding='max_length',
    add_special_tokens=True,
    return_tensors='tf'
)
print(token)


{'input_ids': <tf.Tensor: shape=(1, 256), dtype=int32, numpy=
array([[    0,    57,   222,   545,   312, 23825,   192,  1560,   115,
            2,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,
            1, 

In [23]:
token = tokenizer.convert_ids_to_tokens(token['input_ids'].numpy()[0])
print(token)

['<s>', 'đi', 'học', 'đầy', 'đủ', 'full', 'điểm', 'chuyên', 'cần', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'

In [24]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, comment in enumerate(df['comment']):
        tokenizer_comment = tokenizer.encode_plus(
            comment,
            max_length=256,
            truncation=True,
            padding='max_length',
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenizer_comment['input_ids']
        masks[i, :] = tokenizer_comment['attention_mask']
    return ids, masks

In [25]:
def SentimentDatasetMapFunction(input_ids, attention_mask, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask
    }, labels

In [26]:
train_size = int(0.8 * len(df))
train_data = df[:train_size]
temp_data = df[train_size:]
temp_size = int(0.5 * len(temp_data))
val_data = temp_data[:temp_size]
test_data = temp_data[temp_size:]

In [27]:
X_train_ids = np.zeros((len(train_data), 256))
X_train_masks = np.zeros((len(train_data), 256))
X_train_ids, X_train_masks = generate_training_data(train_data, X_train_ids, X_train_masks, tokenizer)
y_train = np.zeros((len(train_data), 3))
y_train[np.arange(len(train_data)), train_data['label'].values] = 1


X_val_ids = np.zeros((len(val_data), 256))
X_val_masks = np.zeros((len(val_data), 256))
X_val_ids, X_val_masks = generate_training_data(val_data, X_val_ids, X_val_masks, tokenizer)
y_val = np.zeros((len(val_data), 3))
y_val[np.arange(len(val_data)), val_data['label'].values] = 1

X_test_ids = np.zeros((len(test_data), 256))
X_test_masks = np.zeros((len(test_data), 256))
X_test_ids, X_test_masks = generate_training_data(test_data, X_test_ids, X_test_masks, tokenizer)
y_test = np.zeros((len(test_data), 3))
y_test[np.arange(len(test_data)), test_data['label'].values] = 1

In [28]:
train_data = tf.data.Dataset.from_tensor_slices((X_train_ids, X_train_masks, y_train))
val_data = tf.data.Dataset.from_tensor_slices((X_val_ids, X_val_masks, y_val))
test_data = tf.data.Dataset.from_tensor_slices((X_test_ids, X_test_masks, y_test))

train_data = train_data.map(SentimentDatasetMapFunction)
val_data = val_data.map(SentimentDatasetMapFunction)
test_data = test_data.map(SentimentDatasetMapFunction)

In [29]:
batch_size = 128
train_data = train_data.batch(batch_size)
val_data = val_data.batch(batch_size)
test_data = test_data.batch(batch_size)

In [30]:
@register_keras_serializable(package="CustomLayers")
class PhoBERTEmbedding(tf.keras.layers.Layer):
    def __init__(self, model_name="vinai/phobert-base", **kwargs):
        super(PhoBERTEmbedding, self).__init__(**kwargs)
        self.bert = TFAutoModel.from_pretrained(model_name)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        input_ids = tf.cast(input_ids, tf.int32)
        attention_mask = tf.cast(attention_mask, tf.int32)
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]
        return output[:, 0, :]

    def get_config(self):
        config = super().get_config()
        config.update({"model_name": "vinai/phobert-base"})
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [31]:
input_ids = Input(shape=(256,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(256,), dtype=tf.int32, name="attention_mask")

bert_output = PhoBERTEmbedding()([input_ids, attention_mask])

x = Dropout(0.1)(bert_output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.1)(x)
output = Dense(3, activation="softmax")(x)

model = Model(inputs=[input_ids, attention_mask], outputs=output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)    │ (None, 256)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_mask            │ (None, 256)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pho_bert_embedding        │ (None, 768)            │              0 │ input_ids[0][0],       │
│ (PhoBERTEmbedding)        │                        │                │ attention_mask[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 768)            │              0 │ pho_bert_embedding[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 256)            │        196,864 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 256)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 3)              │            771 │ dropout_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 197,635 (772.01 KB)

 Trainable params: 197,635 (772.01 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
epochs = 15
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    batch_size=batch_size
)


Epoch 1/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 212s 2s/step - accuracy: 0.4994 - loss: 0.9766 - val_accuracy: 0.6456 - val_loss: 0.8043
Epoch 2/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 170s 2s/step - accuracy: 0.6332 - loss: 0.7812 - val_accuracy: 0.7409 - val_loss: 0.7093
Epoch 3/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.7189 - loss: 0.6887 - val_accuracy: 0.7737 - val_loss: 0.6489
Epoch 4/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 168s 2s/step - accuracy: 0.7581 - loss: 0.6304 - val_accuracy: 0.7928 - val_loss: 0.6054
Epoch 5/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.7838 - loss: 0.5853 - val_accuracy: 0.8009 - val_loss: 0.5731
Epoch 6/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.7993 - loss: 0.5526 - val_accuracy: 0.8077 - val_loss: 0.5476
Epoch 7/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.8095 - loss: 0.5288 - val_accuracy: 0.8083 - val_loss: 0.5271
Epoch 8/15
102/102 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.8176 - loss: 0.5098 - val_accu

In [ ]:
model.save("phobert_sentiment_model.keras", save_format='tf')

In [ ]:
model = load_model(
    "phobert_sentiment_model.keras")

In [ ]:
y_pred_prob = model.predict([X_test_ids, X_test_masks])
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)

precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

51/51 ━━━━━━━━━━━━━━━━━━━━ 23s 455ms/step
Precision: 0.8488
Recall: 0.8405
F1-score: 0.8194


In [ ]:
def prepare_data(input_texts, tokenizer):
    tokenized_data = tokenizer(
        input_texts,
        max_length=256,
        truncation=True,
        padding='max_length',
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(tokenized_data.input_ids, tf.float32),
        'attention_mask': tf.cast(tokenized_data.attention_mask, tf.float32)
    }


In [ ]:
input_texts = [" Đội ngũ tư vấn viên hỗ trợ nhiệt tình, nhanh chóng. - Giáo trình phù hợp với từng phân cấp, rõ ràng, giúp cải thiện được trình độ của học viên; giáo viên nhiệt tình, gần gũi. - Cơ sở vật chất khang trang, sạch sẽ, học phí phù hợp ngoài ra có nhiều ưu đãi cho học viên"]

In [ ]:
tokenizer_input_text = prepare_data(input_texts, tokenizer)

In [ ]:
import numpy as np
predictions = model.predict(tokenizer_input_text)
predict_label = np.argmax(predictions, axis = 1)
print(predict_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
[2]
